In [90]:
import numpy as np
import matplotlib.pyplot as plt
import math as m
import pandas as pd
from scipy.optimize import curve_fit
from scipy.stats import chisquare
from IPython.display import display, Math
plt.rcParams['figure.max_open_warning'] = 200

In [91]:
t_unit = "s"
v_unit = "mVpp"
T_unit = "K"
f_unit = "Hz"

In [92]:
# # Definition of the function to fit # #
def f(x,a,b):
    res = (-1/a)*x + b
    return res

### User Functions [open only if needed; too much code :) ]

In [93]:
# # Definition of the function to add incertity depending on the scale of the aparata # #
def error(value):
    if value < 1000 and value >= 100:
      sensibility_error = 0.001/m.sqrt(12)
    if value < 100 and value >= 10:
      sensibility_error = 0.0001/m.sqrt(12)
    if value < 10 and value >= 1:
      sensibility_error = 0.00001/m.sqrt(12)
    if value < 1 and value >= 0.1:
      sensibility_error = 0.000001/m.sqrt(12)
    reading_error = 0.0292*value #2.92% of the value
    scale_error = 0.00025*10**3
    err = m.sqrt( (sensibility_error)**2 + (reading_error)**2 + (scale_error)**2 )
    return err


def chidof(obs, exp, sigma, dof):
    obs_arr = np.array(obs)
    exp_arr = np.array(exp)
    sigma_arr = np.array(sigma)
    return sum((obs_arr - exp_arr)**2/sigma_arr**2) / dof


def fitandplot(path, filename, f, initial, plot):
    file = pd.ExcelFile(path+filename)
    sheets = file.sheet_names
    data = pd.read_excel(path+filename, sheet_name=None)

    Q = []
    errQ = []
    T_mean = []
    errT = []
    chisq = []

    for sheet in sheets:
        v = data[sheet]['Voltage']
        v_error = [error(val)/val for val in v]
        v = np.log(v)
#         v = v[7:]
        
        
        t = data[sheet]['Time']
#         t = t[7:]

        T = data[sheet]['T']
        T_value = np.mean(T)
        T_error = abs(T[1]-T[0])/2

        f_0 = data[sheet]['f0'][0]
        errf_0 = (0.003/100)*f_0

        t_unit = "s"
        v_unit = "mVpp"
        T_unit = "K"
        f_unit = "Hz"

        resval,rescov = curve_fit(f, t, v, initial, sigma = v_error)
        reserr = np.sqrt(np.diag(rescov))
        dof = len(v) - len(initial)
        chi_norm = chidof(v, f(t,*resval), v_error, dof)

        # # Calculus of the Q-value # #
        tau = resval[0]
        tau_err = reserr[0]
        Q_value = f_0*tau*m.pi
        Q_error = Q_value*m.sqrt( (tau_err/tau)**2 + (errf_0/f_0)**2 )

        Q.append(Q_value)
        errQ.append(Q_error)
        T_mean.append(T_value)
        errT.append(T_error)
        chisq.append(chi_norm)
        
        if plot == True :
            # # Plot of the data with fit # #
            #sampling time
            h = max([abs((max(t)-min(t))/1000),1])
            fit_time = np.arange(min(t), max(t)+h, h)
            fit_amplitude = f(fit_time, *resval)

            fig = plt.figure(figsize=(6,4), dpi=100);
            fig.suptitle(r"Data from {0} of {1}".format(sheet, filename))
            plt.xlabel(r"$t$ ({0})".format(t_unit), size = 10)
            plt.ylabel(r"$Amplitude$ ({0})".format(v_unit), size = 10)
            plt.plot(t,v,'.',c='k', ms=6)
            plt.errorbar(t, v, yerr=v_error, fmt=".k", capsize=3,alpha = 0.65,label="Data")
            plt.plot(fit_time,fit_amplitude,'--',c='red',label="Fit")

            textstr = '\n'.join((
            r'Q = {0:.2f} $\pm$ {1:.2f}'.format(Q_value,Q_error),
            r'T = {0:.2f} $\pm$ {1:.2f} $\mathrm{{{2}}}$'.format(T_value,T_error,T_unit),
            r'$f_0$ = {0} $\pm$ {1} $\mathrm{{{2}}}$'.format(f_0,errf_0,f_unit),
            r'$\chi^2$/dof = {0:.2f} ({1})'.format(chi_norm, dof)))

            props = dict(boxstyle='square', facecolor='white', alpha=1)

            plt.text(0.6, 0.6, textstr, fontsize=10,
                    verticalalignment='top',transform=fig.transFigure, bbox=props)
            
            plt.legend()
            %config InlineBackend.figure_format='retina'
            plt.tight_layout()
            plt.grid()
            plt.show()

    return Q,errQ,T_mean,errT, chisq

### Analysis

In [94]:
files = ["04_04", "06_04", "13_04", "14_04", "20_04"]
extension = ".xlsx"
path = "C:/Users/Admin/Desktop/3DJoint_DataAnalysis/data/"

In [95]:
data = []
for i in range(len(files)):
    data.append(pd.read_excel(path+files[i]+extension, sheet_name=None))

In [96]:
#This is needed to plot in floating window
%matplotlib qt 

initial = [0.1, 0]

# The function fitandplot takes as input : 
# path, filename, the fitting function, the initial guess list 
# and a boolean variable if you want or not the plots
# Returns 4 arrays containing Q, T and their errors

Q = []
errQ = []
T = []
errT = []
chisq = []


for i in range(len(files)):
    q, errq, t, errt, chi = fitandplot(path, files[i] + extension, f, initial, False)
    Q.extend(q)
    errQ.extend(errq)
    T.extend(t)
    errT.extend(errt)
    chisq.extend(chi)

In [97]:
n, bins, patches = plt.hist(chisq, bins = 10)

In [101]:
# We now plot the results from 04_04 and 06_04
%config InlineBackend.figure_format='retina'
fig = plt.figure(figsize=(6,4), dpi=100);
fig.suptitle('Q vs T')
plt.xlabel(r"$T$ ({0})".format(T_unit), size = 10)
plt.ylabel(r"$Q$",size = 10)

plt.errorbar(T,Q, xerr = errT, yerr = errQ,fmt=".",color = 'r', lw = 1.5,
             alpha = 0.6, capsize=3,label="Data ")


plt.legend()
plt.tight_layout()
plt.grid()
plt.show()

In [39]:
x = np.arange(0.1,1000, 0.1)
y = [error(i) for i in x]
plt.plot(x, x)
plt.fill_between(x, x-y, x+y, alpha = 0.5)